<a href="https://colab.research.google.com/github/ilyx02/ML/blob/main/catboost_lr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ast
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
from numpy.linalg import inv
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error,mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer, OneHotEncoder
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
data = pd.read_csv('/content/train_contest.csv')
a = '/content/train_contest.csv'
с = '/content/for_prediction.csv'
df_a = pd.read_csv(a, sep=',')
df_c = pd.read_csv(с, sep=',')

In [30]:
df_a.head(3)

,premium,name,department,has_test,response_letter_required,area,type,address,response_url,sort_point_distance,...,working_time_intervals,working_time_modes,accept_temporary,description,experience,key_skills,specializations,region,immediate_redirect_url,mean_salary
0,False,Агент по недвижимости,NaN,False,False,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Санкт-Петербург', 'street': 'Невский...",NaN,NaN,...,[],[],False,"<p>Мы ищем людей, которым не достаточно того, ...","{'id': 'noExperience', 'name': 'Нет опыта'}","[{'name': 'Жилая недвижимость'}, {'name': 'Акт...","[{'id': '20.20', 'name': 'Агент', 'profarea_id...",Санкт-Петербург,NaN,125000.0
1,False,IT-специалист/ техник,NaN,False,False,"{'id': '1130', 'name': 'Братск', 'url': 'https...","{'id': 'open', 'name': 'Открытая'}",NaN,NaN,NaN,...,[],[],False,<ul> </ul> <p><em><strong>Навыки</strong></em>...,"{'id': 'between1And3', 'name': 'От 1 года до 3...",[],"[{'id': '1.172', 'name': 'Начальный уровень, М...",Иркутская область,NaN,50000.0
2,False,Старший/ведущий инженер-программист,NaN,False,False,"{'id': '76', 'name': 'Ростов-на-Дону', 'url': ...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Ростов-на-Дону', 'street': '60К-9, 1...",NaN,NaN,...,[],[],False,<strong>Обязанности:</strong> <ul> <li>Разрабо...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'Linux'}, {'name': 'Bash'}, {'name':...","[{'id': '1.221', 'name': 'Программирование, Ра...",Ростовская область,NaN,80000.0


In [32]:
df_c.head(3)

,Id,premium,name,department,has_test,response_letter_required,area,type,address,response_url,...,working_days,working_time_intervals,working_time_modes,accept_temporary,description,experience,key_skills,specializations,region,immediate_redirect_url
0,0,False,Помощник маркетолога,NaN,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': 'open', 'name': 'Открытая'}",NaN,NaN,...,[],[],[],False,<p>В IT-компанию Ищем Интернет-маркетолога (уд...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'Контекстная реклама'}, {'name': 'Ин...","[{'id': '3.206', 'name': 'Печатная реклама', '...",Москва,NaN
1,1,False,Менеджер по персоналу,NaN,False,False,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Санкт-Петербург', 'street': 'Новочер...",NaN,...,[],[],[],False,<p>Мы приглашаем на работу соискателей на долж...,"{'id': 'noExperience', 'name': 'Нет опыта'}","[{'name': 'Подбор персонала'}, {'name': 'Работ...","[{'id': '6.254', 'name': 'Рекрутмент', 'profar...",Санкт-Петербург,NaN
2,2,False,"Инструктор на детскую игровую площадку (ТЦ ""Ме...",NaN,False,False,"{'id': '99', 'name': 'Уфа', 'url': 'https://ap...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Уфа', 'street': 'Рубежная улица', 'b...",NaN,...,[],[],[],False,"<strong>Если ты любишь играть, прыгать и бегат...","{'id': 'noExperience', 'name': 'Нет опыта'}",[],"[{'id': '24.378', 'name': 'Тренерский состав',...",Республика Башкортостан,NaN


In [4]:
import pandas as pd
import ast

def ini(dff):
    # Извлечение имени работодателя
    def extract_employer_name(employer_str):
        employer_dict = ast.literal_eval(employer_str)  # Преобразуем строку в словарь
        return employer_dict['name']

    dff['работодатель'] = dff['employer'].apply(extract_employer_name)

    # Извлечение города
    def extract_city(city_str):
        city_dict = ast.literal_eval(city_str)  # Преобразуем строку в словарь
        return city_dict['name']

    dff['город'] = dff['area'].apply(extract_city)

    # Присвоение значения ценность города
    def assign_city_feature(row):
        city = row['город']
        if city in ['Москва', 'Санкт-Петербург', 'Адлер', 'Сочи', 'Красная Поляна', 'Сириус']:
            return 1  # Лидирующие города
        elif dff['город'].value_counts().get(city, 0) > 100:  # Пример проверки для других условий
            return 0.3  # Если количество больше 100
        else:
            return 0

    dff['city_feature'] = dff.apply(assign_city_feature, axis=1)

    # Извлечение опыта
    def extract_experience(experience_str):
        experience_dict = ast.literal_eval(experience_str)  # Преобразуем строку в словарь
        return experience_dict['name']

    dff['опыт'] = dff['experience'].apply(extract_experience)

    # Классификация профессии
    group_keywords = {
      'Продажа и недвижимость': ['агент','риэлтор', 'продаж', 'недвижимост', 'риелтор', 'аренд'],
      'IT и технические профессии': ['assistant','математ','Ресечер','Technical','Tester','Junior','Сеньор','React','Web','UX','исследователь','dev','OPS','SAP','Scrum','Postgresql','Senior','IT','Scientist','программист', 'администратор', 'инженер', 'технический', 'разработ','ботчик' 'тестировщик', 'develop', 'QA' ,'информационн', 'GameDev', 'administrator', 'програ', 'внедрен', 'HTML', 'DevOps', 'ИТ', 'Data Engineer', 'AI', 'UI', 'интерфей', 'тестир', 'C+', 'game', 'Engineer', 'Backend'],
      'Менеджмент': ['Контролер','Маркетол','Наставник','Методолог','менеджер','Управляющ','Технолог', 'Проджект','рекрутер', 'кадр', 'персонал', 'управление','куратор', 'обучени', 'manager', 'HR', 'мотивац', 'Координатор', 'проектн', 'по ', 'рекрутин'],
      'Дизайн и креатив': ['Таргетолог','Контекстолог','дизайн','Media','UE Artist', 'графический', 'контент', 'SMM', 'Визуализат','Designer','Арт', 'Смм', '2D','3D', 'худож','Живопис'],
      'Аналитика': ['аналит', 'дата', '1C', '1С', 'Бизне', 'Analyst' ,'Product', 'системный','Analytics'],
      'Поддержка пользователей и клиентов': ['поддержк', 'общен', 'Консультант','Оператор', 'абонент', 'Helpdesk' , 'деска', 'клиент'],
      'Спорт': ['Танц', 'тренер', 'тренаж', 'gym', 'физкульт', 'ЛФК', 'спорт', 'Инструктор'],
      'Босс': ['Chief','начальник','Руководит', 'директор', 'Lead', 'Head', 'Брокер','Director', 'Techlead', 'Владелец', 'Техлид', 'эксперт'],
      'Старт_карьеры': ['стажер','Стажёр','Начинаю', 'Техник', 'ученик', 'Помощник'],
      'Забаные_работы': ['(', 'льщик', 'Маст', 'Продавец'],
      'Специлисты' : ['специалист']}

    def classify_profession(profession):
        for group, keywords in group_keywords.items():
            if any(keyword.lower() in profession.lower() for keyword in keywords):
                return group
        return 'Другая'

    dff['profession_group'] = dff['name'].apply(classify_profession)

    # Добавление столбца учет компании на основе квантилей
    def add_company_accounting_column(df):
        # Рассчитываем медианную зарплату для каждой компании
        median_salaries = df.groupby('работодатель')['mean_salary'].median()

        # Рассчитываем 25-й и 75-й квантиль медианных зарплат
        q25 = median_salaries.quantile(0.25)
        q75 = median_salaries.quantile(0.75)

        # Создаем новый столбец 'учет_компании'
        def assign_company_value(employer):
            salary = median_salaries[employer]
            if salary > q75:
                return 1
            elif salary > median_salaries.median():  # между медианой и 75-м квантилем
                return 0.75
            elif salary > q25:  # между 25-м квантилем и медианой
                return 0.5
            else:  # меньше или равно 25-му квантилю
                return 0.25

        df['учет_компании'] = df['работодатель'].map(assign_company_value)

    add_company_accounting_column(dff)  # Вызов функции для добавления столбца

    def extract_schedule(schedule_str):
        schedule_dict = ast.literal_eval(schedule_str)  # Преобразуем строку в словарь
        return schedule_dict.get('name', 'Неизвестно')  # Возвращаем имя графика или "Неизвестно"

    dff['График'] = dff['schedule'].apply(extract_schedule)

    return dff



In [5]:
ini(df_a)

,premium,name,department,has_test,response_letter_required,area,type,address,response_url,sort_point_distance,...,region,immediate_redirect_url,mean_salary,работодатель,город,city_feature,опыт,profession_group,учет_компании,График
0,False,Агент по недвижимости,NaN,False,False,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Санкт-Петербург', 'street': 'Невский...",NaN,NaN,...,Санкт-Петербург,NaN,125000.0,АРИН,Санкт-Петербург,1.0,Нет опыта,Продажа и недвижимость,0.75,Гибкий график
1,False,IT-специалист/ техник,NaN,False,False,"{'id': '1130', 'name': 'Братск', 'url': 'https...","{'id': 'open', 'name': 'Открытая'}",NaN,NaN,NaN,...,Иркутская область,NaN,50000.0,СМП-38,Братск,0.0,От 1 года до 3 лет,IT и технические профессии,0.50,Вахтовый метод
2,False,Старший/ведущий инженер-программист,NaN,False,False,"{'id': '76', 'name': 'Ростов-на-Дону', 'url': ...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Ростов-на-Дону', 'street': '60К-9, 1...",NaN,NaN,...,Ростовская область,NaN,80000.0,"САРМАТ, НПП",Ростов-на-Дону,0.3,От 1 года до 3 лет,IT и технические профессии,0.75,Полный день
3,False,Эксперт / методист приемной комиссии,NaN,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Москва', 'street': '1-я Миусская ули...",NaN,NaN,...,Москва,NaN,120000.0,РХТУ им. Д.И. Менделеева,Москва,1.0,От 1 года до 3 лет,Босс,0.75,Полный день
4,False,Табельщица/табельщик,NaN,False,False,"{'id': '1613', 'name': 'Энгельс', 'url': 'http...","{'id': 'open', 'name': 'Открытая'}","{'city': None, 'street': None, 'building': Non...",NaN,NaN,...,Саратовская область,NaN,15000.0,"Сигнал, группа компаний",Энгельс,0.0,От 1 года до 3 лет,Забаные_работы,0.25,Полный день
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30067,False,Руководитель отдела персонала,NaN,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Москва', 'street': 'Ленинградский пр...",NaN,NaN,...,Москва,NaN,190000.0,СИНЕРГИЯ,Москва,1.0,Более 6 лет,IT и технические профессии,0.75,Полный день
30068,False,Специалист по развитию и обучению персонала,NaN,False,False,"{'id': '1641', 'name': 'Набережные Челны', 'ur...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Набережные Челны', 'street': 'проспе...",NaN,NaN,...,Республика Татарстан,NaN,45000.0,Brinex,Набережные Челны,0.3,От 1 года до 3 лет,IT и технические профессии,0.50,Полный день
30069,False,Инженер-программист,NaN,False,False,"{'id': '1399', 'name': 'Магнитогорск', 'url': ...","{'id': 'open', 'name': 'Открытая'}",NaN,NaN,NaN,...,Челябинская область,NaN,75000.0,Магнитогорский металлургический комбинат,Магнитогорск,0.0,От 1 года до 3 лет,IT и технические профессии,0.50,Полный день
30070,False,Куратор проекта,"{'id': '1947314-1947314-ancor', 'name': 'ANCOR'}",False,False,"{'id': '3', 'name': 'Екатеринбург', 'url': 'ht...","{'id': 'open', 'name': 'Открытая'}",NaN,NaN,NaN,...,Свердловская область,NaN,30000.0,ANCOR,Екатеринбург,0.3,От 1 года до 3 лет,Менеджмент,1.00,Гибкий график


In [6]:
jobs= df_a[['name', 'region', 'работодатель' ,'mean_salary',	'city_feature',	'опыт',  'profession_group','учет_компании','График']]
jobs

# без бустинга для линейной регрессии использовалось get_dummies
#jobs_encoded = pd.get_dummies(jobs, columns=['опыт', 'profession_group','График'])
                             #, drop_first=True)
#jobs_encoded

,name,region,работодатель,mean_salary,city_feature,опыт,profession_group,учет_компании,График
0,Агент по недвижимости,Санкт-Петербург,АРИН,125000.0,1.0,Нет опыта,Продажа и недвижимость,0.75,Гибкий график
1,IT-специалист/ техник,Иркутская область,СМП-38,50000.0,0.0,От 1 года до 3 лет,IT и технические профессии,0.50,Вахтовый метод
2,Старший/ведущий инженер-программист,Ростовская область,"САРМАТ, НПП",80000.0,0.3,От 1 года до 3 лет,IT и технические профессии,0.75,Полный день
3,Эксперт / методист приемной комиссии,Москва,РХТУ им. Д.И. Менделеева,120000.0,1.0,От 1 года до 3 лет,Босс,0.75,Полный день
4,Табельщица/табельщик,Саратовская область,"Сигнал, группа компаний",15000.0,0.0,От 1 года до 3 лет,Забаные_работы,0.25,Полный день
...,...,...,...,...,...,...,...,...,...
30067,Руководитель отдела персонала,Москва,СИНЕРГИЯ,190000.0,1.0,Более 6 лет,IT и технические профессии,0.75,Полный день
30068,Специалист по развитию и обучению персонала,Республика Татарстан,Brinex,45000.0,0.3,От 1 года до 3 лет,IT и технические профессии,0.50,Полный день
30069,Инженер-программист,Челябинская область,Магнитогорский металлургический комбинат,75000.0,0.0,От 1 года до 3 лет,IT и технические профессии,0.50,Полный день
30070,Куратор проекта,Свердловская область,ANCOR,30000.0,0.3,От 1 года до 3 лет,Менеджмент,1.00,Гибкий график


In [7]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.1 MB/s eta 0:00:00


In [11]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, r2_score


In [ ]:
jobs_encoded['name'] = jobs_encoded['name'].astype('category')
jobs_encoded['region'] = jobs_encoded['region'].astype('category')
jobs_encoded['работодатель'] = jobs_encoded['работодатель'].astype('category')

# Определение целевой переменной и признаков
X = jobs_encoded.drop(columns=['mean_salary'])  # Признаки
y = jobs_encoded['mean_salary']  # Целевая переменная

# Определение категориальных признаков
categorical_features = ['name', 'region', 'работодатель']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Инициализация CatBoostRegressor
model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, cat_features=categorical_features, verbose=200)

# Обучение модели
model.fit(X_train, np.log(y_train))

# Прогнозирование на тестовой выборке
y_pred = model.predict(X_test)
print('MAE (тестовая выборка):', mae(y_test, np.exp(y_pred)))
print('R² (тестовая выборка):', r2_score(y_test, np.exp(y_pred)))

In [12]:
jobs['name'] = jobs['name'].astype('category')
jobs['region'] = jobs['region'].astype('category')
jobs['работодатель'] = jobs['работодатель'].astype('category')
jobs['опыт'] = jobs['опыт'].astype('category')
jobs['profession_group'] = jobs['profession_group'].astype('category')
jobs['График'] = jobs['График'].astype('category')

# Определение целевой переменной и признаков
X = jobs.drop(columns=['mean_salary'])  # Признаки
y = jobs['mean_salary']  # Целевая переменная

# Определение категориальных признаков
categorical_features = ['name', 'region', 'работодатель','опыт',  'profession_group','График']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Инициализация CatBoostRegressor
model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, cat_features=categorical_features, verbose=200)

# Обучение модели
model.fit(X_train, np.log(y_train))

# Прогнозирование на тестовой выборке
y_pred = model.predict(X_test)
print('MAE (тестовая выборка):', mean_absolute_error(y_test, np.exp(y_pred)))
print('R² (тестовая выборка):', r2_score(y_test, np.exp(y_pred)))

<ipython-input-12-bae2cd9fa253>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs['name'] = jobs['name'].astype('category')
<ipython-input-12-bae2cd9fa253>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs['region'] = jobs['region'].astype('category')
<ipython-input-12-bae2cd9fa253>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

0:	learn: 0.7261024	total: 40.5ms	remaining: 40.4s
200:	learn: 0.3618718	total: 5.07s	remaining: 20.2s
400:	learn: 0.3443108	total: 10.6s	remaining: 15.8s
600:	learn: 0.3345866	total: 15.3s	remaining: 10.2s
800:	learn: 0.3271546	total: 19.6s	remaining: 4.86s
999:	learn: 0.3203907	total: 25.5s	remaining: 0us
MAE (тестовая выборка): 26188.90439243902
R² (тестовая выборка): 0.568699677620333


In [13]:
print('MAE (тестовая выборка):', mean_absolute_error(y_test, np.exp(y_pred)))
print('R² (тестовая выборка):', r2_score(y_test, np.exp(y_pred)))

MAE (тестовая выборка): 26188.90439243902
R² (тестовая выборка): 0.568699677620333


# Предсказания

In [25]:
salary_df = df_a[['работодатель', 'учет_компании']].drop_duplicates()
print(salary_df.head(5))
#создаем словарь с рейтингами исходят из тренирвочоного датасета
company_ratings = salary_df.set_index('работодатель')['учет_компании'].to_dict()
print('число компаний - ', len(company_ratings))

               работодатель  учет_компании
0                      АРИН           0.75
1                    СМП-38           0.50
2               САРМАТ, НПП           0.75
3  РХТУ им. Д.И. Менделеева           0.75
4   Сигнал, группа компаний           0.25
число компаний -  12973


In [26]:
import pandas as pd
import ast

def inia(dff, salary_df):
    # Извлечение имени работодателя
    def extract_employer_name(employer_str):
        employer_dict = ast.literal_eval(employer_str)  # Преобразуем строку в словарь
        return employer_dict['name']

    dff['работодатель'] = dff['employer'].apply(extract_employer_name)

    # Извлечение города
    def extract_city(city_str):
        city_dict = ast.literal_eval(city_str)  # Преобразуем строку в словарь
        return city_dict['name']

    dff['город'] = dff['area'].apply(extract_city)

    # Присвоение значения city_feature
    def assign_city_feature(row):
        city = row['город']
        if city in ['Москва', 'Санкт-Петербург', 'Адлер', 'Сочи']:
            return 1  # Лидирующие города
        elif dff['город'].value_counts().get(city, 0) > 100:  # Пример проверки для других условий
            return 0.5  # Если количество больше 100
        else:
            return 0

    dff['city_feature'] = dff.apply(assign_city_feature, axis=1)

    # Извлечение опыта
    def extract_experience(experience_str):
        experience_dict = ast.literal_eval(experience_str)  # Преобразуем строку в словарь
        return experience_dict['name']

    dff['опыт'] = dff['experience'].apply(extract_experience)

    # Классификация профессии
    group_keywords = {
      'Продажа и недвижимость': ['агент','риэлтор', 'продаж', 'недвижимост', 'риелтор', 'аренд'],
      'IT и технические профессии': ['assistant','математ','Ресечер','Technical','Tester','Junior','Сеньор','React','Web','UX','исследователь','dev','OPS','SAP','Scrum','Postgresql','Senior','IT','Scientist','программист', 'администратор', 'инженер', 'технический', 'разработ','ботчик' 'тестировщик', 'develop', 'QA' ,'информационн', 'GameDev', 'administrator', 'програ', 'внедрен', 'HTML', 'DevOps', 'ИТ', 'Data Engineer', 'AI', 'UI', 'интерфей', 'тестир', 'C+', 'game', 'Engineer', 'Backend'],
      'Менеджмент': ['Контролер','Маркетол','Наставник','Методолог','менеджер','Управляющ','Технолог', 'Проджект','рекрутер', 'кадр', 'персонал', 'управление','куратор', 'обучени', 'manager', 'HR', 'мотивац', 'Координатор', 'проектн', 'по ', 'рекрутин'],
      'Дизайн и креатив': ['Таргетолог','Контекстолог','дизайн','Media','UE Artist', 'графический', 'контент', 'SMM', 'Визуализат','Designer','Арт', 'Смм', '2D','3D', 'худож','Живопис'],
      'Аналитика': ['аналит', 'дата', '1C', '1С', 'Бизне', 'Analyst' ,'Product', 'системный','Analytics'],
     'Поддержка пользователей и клиентов': ['поддержк', 'общен', 'Консультант','Оператор', 'абонент', 'Helpdesk' , 'деска', 'клиент'],
      'Спорт': ['Танц', 'тренер', 'тренаж', 'gym', 'физкульт', 'ЛФК', 'спорт', 'Инструктор'],
      'Босс': ['Chief','начальник','Руководит', 'директор', 'Lead', 'Head', 'Брокер','Director', 'Techlead', 'Владелец', 'Техлид', 'эксперт'],
      'Старт_карьеры': ['стажер','Стажёр','Начинаю', 'Техник', 'ученик', 'Помощник'],
      'Забаные_работы': ['(', 'льщик', 'Маст', 'Продавец'],
      'Специлисты' : ['специалист']}

    def classify_profession(profession):
        for group, keywords in group_keywords.items():
            if any(keyword.lower() in profession.lower() for keyword in keywords):
                return group
        return 'Другая'

    dff['profession_group'] = dff['name'].apply(classify_profession)

    # Добавление столбца учет компании на основе квантилей из salary_df
    def add_company_accounting_column(df, salary_df):
        # Создаем словарь с рейтингами компаний из salary_df
        company_ratings = salary_df.set_index('работодатель')['учет_компании'].to_dict()

        # Функция для присвоения значения 'учет_компании'
        def assign_company_value(employer):
            return company_ratings.get(employer, 0)  # Если нет данных о зарплате для этой компании, возвращаем 0

        # Применяем функцию к столбцу 'работодатель'
        df['учет_компании'] = df['работодатель'].map(assign_company_value)

    add_company_accounting_column(dff, salary_df)  # Вызов функции для добавления столбца

    # Добавление столбца График
    def extract_schedule(schedule_str):
        schedule_dict = ast.literal_eval(schedule_str)  # Преобразуем строку в словарь
        return schedule_dict.get('name', 'Неизвестно')  # Возвращаем имя графика или "Неизвестно"

    dff['График'] = dff['schedule'].apply(extract_schedule)

    return dff



In [33]:
inia(df_c, salary_df)

,Id,premium,name,department,has_test,response_letter_required,area,type,address,response_url,...,specializations,region,immediate_redirect_url,работодатель,город,city_feature,опыт,profession_group,учет_компании,График
0,0,False,Помощник маркетолога,NaN,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': 'open', 'name': 'Открытая'}",NaN,NaN,...,"[{'id': '3.206', 'name': 'Печатная реклама', '...",Москва,NaN,EKO BIKE,Москва,1.0,От 1 года до 3 лет,Менеджмент,0.25,Удаленная работа
1,1,False,Менеджер по персоналу,NaN,False,False,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Санкт-Петербург', 'street': 'Новочер...",NaN,...,"[{'id': '6.254', 'name': 'Рекрутмент', 'profar...",Санкт-Петербург,NaN,"Ваш дом, агентство недвижимости",Санкт-Петербург,1.0,Нет опыта,Менеджмент,0.00,Полный день
2,2,False,"Инструктор на детскую игровую площадку (ТЦ ""Ме...",NaN,False,False,"{'id': '99', 'name': 'Уфа', 'url': 'https://ap...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Уфа', 'street': 'Рубежная улица', 'b...",NaN,...,"[{'id': '24.378', 'name': 'Тренерский состав',...",Республика Башкортостан,NaN,Мегаполис,Уфа,0.5,Нет опыта,Спорт,0.50,Полный день
3,3,False,Программист Delphi,NaN,False,False,"{'id': '4', 'name': 'Новосибирск', 'url': 'htt...","{'id': 'open', 'name': 'Открытая'}",NaN,NaN,...,"[{'id': '1.221', 'name': 'Программирование, Ра...",Новосибирская область,NaN,AMS Software,Новосибирск,0.5,От 3 до 6 лет,IT и технические профессии,0.75,Удаленная работа
4,4,False,Ведущий специалист ВКС,NaN,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Москва', 'street': 'Чистопрудный бул...",NaN,...,"[{'id': '1.172', 'name': 'Начальный уровень, М...",Москва,NaN,Технокад,Москва,1.0,От 1 года до 3 лет,Специлисты,0.00,Полный день
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7513,7513,False,HR менеджер,NaN,False,False,"{'id': '131', 'name': 'Симферополь', 'url': 'h...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Симферополь', 'street': 'улица Жени ...",NaN,...,"[{'id': '6.254', 'name': 'Рекрутмент', 'profar...",Республика Крым,NaN,Группа компаний Shels (ООО Шелс),Симферополь,0.0,От 1 года до 3 лет,Менеджмент,0.25,Полный день
7514,7514,False,Руководитель HR отдела,NaN,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Москва', 'street': 'Электрический пе...",NaN,...,"[{'id': '6.254', 'name': 'Рекрутмент', 'profar...",Москва,NaN,Edison,Москва,1.0,От 1 года до 3 лет,IT и технические профессии,0.75,Полный день
7515,7515,False,Системный аналитик (Стажер),"{'id': '80-80-bank', 'name': ' Альфа-Банк'}",False,False,"{'id': '3', 'name': 'Екатеринбург', 'url': 'ht...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Екатеринбург', 'street': 'Горького',...",NaN,...,"[{'id': '17.751', 'name': 'Другое', 'profarea_...",Свердловская область,NaN,Альфа-Банк,Екатеринбург,0.5,Нет опыта,IT и технические профессии,0.75,Гибкий график
7516,7516,False,Руководитель Отдела развития банковских технол...,NaN,False,False,"{'id': '22', 'name': 'Владивосток', 'url': 'ht...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Владивосток', 'street': 'Партизански...",NaN,...,"[{'id': '1.221', 'name': 'Программирование, Ра...",Приморский край,NaN,СКБ Приморья Примсоцбанк,Владивосток,0.0,От 1 года до 3 лет,IT и технические профессии,1.00,Полный день


In [34]:
jobs_pred = df_c[['name', 'region', 'работодатель' ,	'city_feature',	'опыт',  'profession_group','учет_компании','График']]
jobs_pred

,name,region,работодатель,city_feature,опыт,profession_group,учет_компании,График
0,Помощник маркетолога,Москва,EKO BIKE,1.0,От 1 года до 3 лет,Менеджмент,0.25,Удаленная работа
1,Менеджер по персоналу,Санкт-Петербург,"Ваш дом, агентство недвижимости",1.0,Нет опыта,Менеджмент,0.00,Полный день
2,"Инструктор на детскую игровую площадку (ТЦ ""Ме...",Республика Башкортостан,Мегаполис,0.5,Нет опыта,Спорт,0.50,Полный день
3,Программист Delphi,Новосибирская область,AMS Software,0.5,От 3 до 6 лет,IT и технические профессии,0.75,Удаленная работа
4,Ведущий специалист ВКС,Москва,Технокад,1.0,От 1 года до 3 лет,Специлисты,0.00,Полный день
...,...,...,...,...,...,...,...,...
7513,HR менеджер,Республика Крым,Группа компаний Shels (ООО Шелс),0.0,От 1 года до 3 лет,Менеджмент,0.25,Полный день
7514,Руководитель HR отдела,Москва,Edison,1.0,От 1 года до 3 лет,IT и технические профессии,0.75,Полный день
7515,Системный аналитик (Стажер),Свердловская область,Альфа-Банк,0.5,Нет опыта,IT и технические профессии,0.75,Гибкий график
7516,Руководитель Отдела развития банковских технол...,Приморский край,СКБ Приморья Примсоцбанк,0.0,От 1 года до 3 лет,IT и технические профессии,1.00,Полный день


In [35]:
jobs_pred['name'] = jobs_pred['name'].astype('category')
jobs_pred['region'] = jobs_pred['region'].astype('category')
jobs_pred['работодатель'] = jobs_pred['работодатель'].astype('category')
jobs_pred['опыт'] = jobs_pred['опыт'].astype('category')
jobs_pred['profession_group'] = jobs_pred['profession_group'].astype('category')
jobs_pred['График'] = jobs_pred['График'].astype('category')
jobs_pred

<ipython-input-35-8355428fcea4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs_pred['name'] = jobs_pred['name'].astype('category')
<ipython-input-35-8355428fcea4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs_pred['region'] = jobs_pred['region'].astype('category')
<ipython-input-35-8355428fcea4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

,name,region,работодатель,city_feature,опыт,profession_group,учет_компании,График
0,Помощник маркетолога,Москва,EKO BIKE,1.0,От 1 года до 3 лет,Менеджмент,0.25,Удаленная работа
1,Менеджер по персоналу,Санкт-Петербург,"Ваш дом, агентство недвижимости",1.0,Нет опыта,Менеджмент,0.00,Полный день
2,"Инструктор на детскую игровую площадку (ТЦ ""Ме...",Республика Башкортостан,Мегаполис,0.5,Нет опыта,Спорт,0.50,Полный день
3,Программист Delphi,Новосибирская область,AMS Software,0.5,От 3 до 6 лет,IT и технические профессии,0.75,Удаленная работа
4,Ведущий специалист ВКС,Москва,Технокад,1.0,От 1 года до 3 лет,Специлисты,0.00,Полный день
...,...,...,...,...,...,...,...,...
7513,HR менеджер,Республика Крым,Группа компаний Shels (ООО Шелс),0.0,От 1 года до 3 лет,Менеджмент,0.25,Полный день
7514,Руководитель HR отдела,Москва,Edison,1.0,От 1 года до 3 лет,IT и технические профессии,0.75,Полный день
7515,Системный аналитик (Стажер),Свердловская область,Альфа-Банк,0.5,Нет опыта,IT и технические профессии,0.75,Гибкий график
7516,Руководитель Отдела развития банковских технол...,Приморский край,СКБ Приморья Примсоцбанк,0.0,От 1 года до 3 лет,IT и технические профессии,1.00,Полный день


In [39]:
y_predict = model.predict(jobs_pred)
y_predict = np.exp(y_predict)
data_test = { 'Predicted': y_predict}
df_test = pd.DataFrame(data_test).reset_index()
df_test = df_test.rename(columns={"index":"Id"})
df_test

,Id,Predicted
0,0,27003.246655
1,1,31613.702862
2,2,41805.723619
3,3,115786.354471
4,4,38032.144848
...,...,...
7513,7513,33018.956767
7514,7514,83010.112325
7515,7515,79428.826222
7516,7516,103165.891759


In [40]:
df_test.to_csv("predict_kaggle_Safiullin.csv", index=False)